# Controllable generation via RL to let Elon Musk speak ill of DOGE
> How to control text generation through a sentiment classifier.



In [1]:
%pip install pfrl@git+https://github.com/voidful/pfrl.git
%pip install textrl==0.2.15

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/voidful/pfrl.git to /tmp/pip-install-du1ubgh_/pfrl_d7f21da864944d2c8f2a8f537dfcb8c8
  Running command git clone --filter=blob:none -q https://github.com/voidful/pfrl.git /tmp/pip-install-du1ubgh_/pfrl_d7f21da864944d2c8f2a8f537dfcb8c8
  Resolved https://github.com/voidful/pfrl.git to commit bd679022d91902a3c88066d2646ad69b30d5684a
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 721 kB 1.1 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pfrl: filename=pfrl-0.4.0-py3-none-any.whl size=155504 sha256=950dad1f36449504d8145a709b5e06e834a8d5f77b00701da21aeb35f6be4d6e
  Stored in directory: /tmp/pip-ephem-wheel-cache-ue8da_4w/wheels/4e/7a/0d/6d90ea0eecefee2bcf373177932fd54e84b41ef5190472e2db
  Created wheel for gym: filename=gym-0.26.2-py3-

In [2]:
from textrl import TextRLEnv,TextRLActor
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelWithLMHead
import logging
import sys
import pfrl
import torch
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

**Using a pre-trained model, it can generate elonmusk's style tweets.**

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans")
model = AutoModelForSeq2SeqLM.from_pretrained("lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans")

# tokenizer = AutoTokenizer.from_pretrained("huggingtweets/elonmusk")  
# model = AutoModelWithLMHead.from_pretrained("huggingtweets/elonmusk")
model.eval()
model.cuda()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(59481, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(59481, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

**a sentiment classifier for rl reward**

In [4]:
sentiment = pipeline('sentiment-analysis',model="cardiffnlp/twitter-roberta-base-sentiment",tokenizer="cardiffnlp/twitter-roberta-base-sentiment",device=0,return_all_scores=True)

/home/b0990106x/miniconda3/envs/textrl/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [5]:
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.CRITICAL)

In [6]:
sentiment("dogecoin is bad")

[[{'label': 'LABEL_0', 'score': 0.9338533878326416},
  {'label': 'LABEL_1', 'score': 0.0601188987493515},
  {'label': 'LABEL_2', 'score': 0.006027725990861654}]]

In [7]:
sentiment("dogecoin is bad")[0][0]['score']

0.9338533878326416

set our text generation reward, inverse perplexity + sentiment classifier.
- inverse perplexity make sure the generated sentence probability will be high.
- sentiment classifier can make the generate more negative.

In [8]:
class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish): # predicted will be the list of predicted token
      reward = 0
      if finish or len(predicted_list) >= self.env_max_length:
        predicted_text = tokenizer.convert_tokens_to_string(predicted_list[0])
        # sentiment classifier
        reward = sentiment(input_item[0]+predicted_text)[0][0]['score'] * 10
      return reward

**fit one example**

In [9]:
observaton_list = [{'input':'i think dogecoin is'}]

In [10]:
env = MyRLEnv(model, tokenizer, observation_input=observaton_list,compare_sample=1)
actor = TextRLActor(env,model,tokenizer)
agent = actor.agent_ppo(update_interval=100, minibatch_size=3, epochs=10)

In [11]:
actor.predict(observaton_list[0])

['<s>v_tok_2201v_tok_4870v_tok_4689v_tok_4038v_tok_4689v_tok_4038v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689']

In [12]:
pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=300,
    eval_n_steps=None,
    eval_n_episodes=1,       
    train_max_episode_len=100,  
    eval_interval=10,
    outdir='elon_musk_dogecoin', 
)

/home/b0990106x/miniconda3/envs/textrl/lib/python3.9/site-packages/pfrl/agents/ppo.py:132: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180549130/work/torch/csrc/utils/tensor_new.cpp:201.)
  actions = torch.tensor([b["action"] for b in dataset], device=device)
/home/b0990106x/miniconda3/envs/textrl/lib/python3.9/site-packages/textrl/actor.py:293: UserWarning: Using a target size (torch.Size([3, 1])) that is different to the input size (torch.Size([3, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss_value_func = F.mse_loss(vs_pred, vs_teacher)


outdir:elon_musk_dogecoin step:100 episode:0 R:0
statistics:[('average_value', -0.0058390973), ('average_entropy', 7.394447), ('average_value_loss', 0.0037143150047631935), ('average_policy_loss', 0.005632610293105245), ('n_updates', 334), ('explained_variance', -14.406542544507479)]


evaluation episode 0 length:100 R:0
The best score is updated -3.4028235e+38 -> 0.0
Saved the agent to elon_musk_dogecoin/best
outdir:elon_musk_dogecoin step:200 episode:1 R:0
statistics:[('average_value', 0.007455408), ('average_entropy', 7.2373095), ('average_value_loss', 0.004107499678502791), ('average_policy_loss', 0.0016428552218712867), ('n_updates', 668), ('explained_variance', -51.67072420067641)]
evaluation episode 0 length:100 R:0
outdir:elon_musk_dogecoin step:300 episode:2 R:0
statistics:[('average_value', -0.0097230775), ('average_entropy', 7.222486), ('average_value_loss', 0.0031916120633104584), ('average_policy_loss', -0.008724038759246468), ('n_updates', 1002), ('explained_variance', -13.687780137957585)]
evaluation episode 0 length:100 R:0
Saved the agent to elon_musk_dogecoin/300_finish


(<textrl.actor.TextPPO at 0x7f2883bdbee0>,
 [{'average_value': -0.0058390973,
   'average_entropy': 7.394447,
   'average_value_loss': 0.0037143150047631935,
   'average_policy_loss': 0.005632610293105245,
   'n_updates': 334,
   'explained_variance': -14.406542544507479,
   'eval_score': 0.0},
  {'average_value': 0.007455408,
   'average_entropy': 7.2373095,
   'average_value_loss': 0.004107499678502791,
   'average_policy_loss': 0.0016428552218712867,
   'n_updates': 668,
   'explained_variance': -51.67072420067641,
   'eval_score': 0.0},
  {'average_value': -0.0097230775,
   'average_entropy': 7.222486,
   'average_value_loss': 0.0031916120633104584,
   'average_policy_loss': -0.008724038759246468,
   'n_updates': 1002,
   'explained_variance': -13.687780137957585,
   'eval_score': 0.0}])

loading the best result and predict.

In [13]:
agent.load("./elon_musk_dogecoin/best")

In [14]:
actor.predict(observaton_list[0])

['<s>v_tok_2201v_tok_4870v_tok_4689v_tok_4038v_tok_4689v_tok_4038v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689v_tok_4689']